<img src="https://certificate.tpq.io/taim_logo.png" width="350px" align="right">

# Artificial Intelligence in Finance

## Execution & Deployment

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

In [ ]:
import os
import time
import numpy as np
import pandas as pd
from pprint import pprint
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', '{:.5f}'.format)
np.set_printoptions(suppress=True, precision=4)
os.environ['PYTHONHASHSEED'] = '0'

## Oanda Account

``pip install --upgrade git+https://github.com/yhilpisch/tpqoa.git``

Configuration file ``aiif.cfg``:

    [oanda]
    account_id = XYZ-ABC-...
    access_token = ZYXCAB...
    account_type = practice

In [ ]:
import tpqoa

In [ ]:
api = tpqoa.tpqoa('../aiif.cfg')

In [ ]:
ins = api.get_instruments()

In [ ]:
ins[:5]

## Data Retrieval 

In [ ]:
raw = api.get_history(instrument='EUR_USD',
                      start='2018-01-01',
                      end='2020-07-31',
                      granularity='D',
                      price='A')

In [ ]:
raw.info()

In [ ]:
raw.head()

In [ ]:
raw['c'].plot(figsize=(10, 6));

In [ ]:
%%time
raw = api.get_history(instrument='EUR_USD',
                      start='2020-07-01',
                      end='2020-07-31',
                      granularity='M1',
                      price='M') 

In [ ]:
raw.info()

In [ ]:
raw.tail()

In [ ]:
raw['c'].plot(figsize=(10, 6));

In [ ]:
api.stream_data('EUR_USD', stop=10)

## Order Execution 

In [ ]:
order = api.create_order('EUR_USD', units=25000,
                         suppress=True, ret=True)
pprint(order)

In [ ]:
def print_details(order):
    details = (order['time'][:-7], order['instrument'], order['units'],
               order['price'], order['pl'])
    return details

In [ ]:
print_details(order)

In [ ]:
time.sleep(1)

In [ ]:
order = api.create_order('EUR_USD', units=-25000,
                         suppress=True, ret=True)
pprint(order)

In [ ]:
print_details(order)

In [ ]:
time.sleep(1)

In [ ]:
order = api.create_order('EUR_USD', units=25000,
                         sl_distance=0.005,
                         suppress=True, ret=True)

In [ ]:
print_details(order)

In [ ]:
sl_order = api.get_transaction(tid=int(order['id']) + 1)

In [ ]:
sl_order

In [ ]:
(sl_order['time'], sl_order['type'], order['price'],
 sl_order['price'], sl_order['distance'])

In [ ]:
time.sleep(1)

In [ ]:
order = api.create_order('EUR_USD', units=-25000, suppress=True, ret=True)

In [ ]:
print_details(order)

In [ ]:
order = api.create_order('EUR_USD', units=25000,
                         tsl_distance=0.005,
                         suppress=True, ret=True)

In [ ]:
print_details(order)

In [ ]:
tsl_order = api.get_transaction(tid=int(order['id']) + 1)

In [ ]:
tsl_order

In [ ]:
(tsl_order['time'][:-7], tsl_order['type'],
 order['price'], tsl_order['distance'])

In [ ]:
time.sleep(1)

In [ ]:
order = api.create_order('EUR_USD', units=-25000,
                         suppress=True, ret=True)

In [ ]:
print_details(order)

In [ ]:
time.sleep(1)

In [ ]:
tp_price = round(order['price'] + 0.005, 4)
tp_price

In [ ]:
order = api.create_order('EUR_USD', units=25000,
                         tp_price=tp_price,
                         suppress=True, ret=True)

In [ ]:
print_details(order)

In [ ]:
tp_order = api.get_transaction(tid=int(order['id']) + 1)

In [ ]:
tp_order

In [ ]:
(tp_order['time'][:-7], tp_order['type'],
 order['price'], tp_order['price'])

In [ ]:
time.sleep(1)

In [ ]:
order = api.create_order('EUR_USD', units=-25000,
                         suppress=True, ret=True)

In [ ]:
print_details(order)

In [ ]:
api.print_transactions(tid=int(order['id']) - 22)

In [ ]:
api.get_account_summary()

## Trading Bot

In [ ]:
import oandaenv as oe

In [ ]:
symbol = 'EUR_USD'

In [ ]:
date = '2020-11-19'  # adjust date to a recent date

In [ ]:
features = [symbol, 'r', 's', 'm', 'v']

In [ ]:
%%time
learn_env = oe.OandaEnv(symbol=symbol,
                  start=f'{date} 08:00:00',
                  end=f'{date} 13:00:00',
                  granularity='S30',
                  price='M',
                  features=features,
                  window=20,
                  lags=3,
                  leverage=20,
                  min_accuracy=0.4,
                  min_performance=0.85
                 )

In [ ]:
np.bincount(learn_env.data['d'])

In [ ]:
learn_env.data.info()

In [ ]:
valid_env = oe.OandaEnv(symbol=learn_env.symbol,
                  start=f'{date} 13:00:00',
                  end=f'{date} 14:00:00',
                  granularity=learn_env.granularity,
                  price=learn_env.price,
                  features=learn_env.features,
                  window=learn_env.window,
                  lags=learn_env.lags,
                  leverage=learn_env.leverage,
                  min_accuracy=0,
                  min_performance=0,
                  mu=learn_env.mu,
                  std=learn_env.std
                 )

In [ ]:
valid_env.data.info()

In [ ]:
test_env = oe.OandaEnv(symbol=learn_env.symbol,
                  start=f'{date} 14:00:00',
                  end=f'{date} 17:00:00',
                  granularity=learn_env.granularity,
                  price=learn_env.price,
                  features=learn_env.features,
                  window=learn_env.window,
                  lags=learn_env.lags,
                  leverage=learn_env.leverage,
                  min_accuracy=0,
                  min_performance=0,
                  mu=learn_env.mu,
                  std=learn_env.std
                 )

In [ ]:
test_env.data.info()

In [ ]:
ax = learn_env.data[learn_env.symbol].plot(figsize=(10, 6))
plt.axvline(learn_env.data.index[-1], ls='--')
valid_env.data[learn_env.symbol].plot(ax=ax, style='-.')
plt.axvline(valid_env.data.index[-1], ls='--')
test_env.data[learn_env.symbol].plot(ax=ax, style='-.');

In [ ]:
import tradingbot

In [ ]:
tradingbot.set_seeds(100)
agent = tradingbot.TradingBot(24, 0.001, learn_env=learn_env,
                              valid_env=valid_env)

In [ ]:
episodes = 31

In [ ]:
%time agent.learn(episodes)

In [ ]:
tradingbot.plot_performance(agent)

In [ ]:
import backtest as bt

In [ ]:
env = test_env

In [ ]:
bt.backtest(agent, env)

In [ ]:
env.data['p'].iloc[env.lags:].value_counts()

In [ ]:
sum(env.data['p'].iloc[env.lags:].diff() != 0)

In [ ]:
(env.data[['r', 's']].iloc[env.lags:] * env.leverage).sum(
        ).apply(np.exp)

In [ ]:
(env.data[['r', 's']].iloc[env.lags:] * env.leverage).sum(
        ).apply(np.exp) - 1

In [ ]:
(env.data[['r', 's']].iloc[env.lags:] * env.leverage).cumsum(
        ).apply(np.exp).plot(figsize=(10, 6));

## Deployment

In [ ]:
import tpqoa

In [ ]:
class OandaTradingBot(tpqoa.tpqoa):
    def __init__(self, config_file, agent, granularity, units,
                 verbose=True):
        super(OandaTradingBot, self).__init__(config_file)
        self.agent = agent
        self.symbol = self.agent.learn_env.symbol
        self.env = agent.learn_env
        self.window = self.env.window
        if granularity is None:
            self.granularity = agent.learn_env.granularity
        else:
            self.granularity = granularity
        self.units = units
        self.trades = 0
        self.position = 0
        self.tick_data = pd.DataFrame()
        self.min_length = (self.agent.learn_env.window +
                           self.agent.learn_env.lags)
        self.pl = list()
        self.verbose = verbose
    def _prepare_data(self):
        self.data['r'] = np.log(self.data / self.data.shift(1))
        self.data.dropna(inplace=True)
        self.data['s'] = self.data[self.symbol].rolling(
                                            self.window).mean()
        self.data['m'] = self.data['r'].rolling(self.window).mean()
        self.data['v'] = self.data['r'].rolling(self.window).std()
        self.data.dropna(inplace=True)
        # self.data_ = (self.data - self.env.mu) / self.env.std  # "correct" approach
        self.data_ = (self.data - self.data.mean()) / self.data.std()  # for demonstration
    def _resample_data(self):
        self.data = self.tick_data.resample(self.granularity,
                        label='right').last().ffill().iloc[:-1]
        self.data = pd.DataFrame(self.data['mid'])
        self.data.columns = [self.symbol,]
        self.data.index = self.data.index.tz_localize(None)
    def _get_state(self):
        state = self.data_[self.env.features].iloc[-self.env.lags:]
        return np.reshape(state.values, [1, self.env.lags,
                                         self.env.n_features])
    def report_trade(self, time, side, order):
        self.trades += 1
        pl = float(order['pl'])
        self.pl.append(pl)
        cpl = sum(self.pl)
        print('\n' + 75 * '=')
        print(f'{time} | *** GOING {side} ({self.trades}) ***')
        print(f'{time} | PROFIT/LOSS={pl:.2f} | CUMULATIVE={cpl:.2f}')
        print(75 * '=')
        if self.verbose:
            pprint(order)
            print(75 * '=')
    def on_success(self, time, bid, ask):
        df = pd.DataFrame({'ask': ask, 'bid': bid,
                           'mid': (bid + ask) / 2},
                          index=[pd.Timestamp(time)])
        self.tick_data = self.tick_data.append(df)
        self._resample_data()
        if len(self.data) > self.min_length:
            self.min_length += 1
            self._prepare_data()
            state = self._get_state()
            prediction = np.argmax(
                self.agent.model.predict(state)[0, 0])
            position = 1 if prediction == 1 else -1
            if self.position in [0, -1] and position == 1:
                order = self.create_order(self.symbol,
                        units=(1 - self.position) * self.units,
                                suppress=True, ret=True)
                self.report_trade(time, 'LONG', order)
                self.position = 1
            elif self.position in [0, 1] and position == -1:
                order = self.create_order(self.symbol,
                        units=-(1 + self.position) * self.units,
                                suppress=True, ret=True)
                self.report_trade(time, 'SHORT', order)
                self.position = -1

In [ ]:
otb = OandaTradingBot('../aiif.cfg', agent, '3s',
                      25000, verbose=False)

In [ ]:
otb.tick_data.info()

In [ ]:
%time otb.stream_data(agent.learn_env.symbol, stop=1000)

In [ ]:
print('\n' + 75 * '=')
print('*** CLOSING OUT ***')
try:
    order = otb.create_order(otb.symbol,
                    units=-otb.position * otb.units,
                    suppress=True, ret=True)
    otb.report_trade(otb.time, 'NEUTRAL', order)
    if otb.verbose:
        pprint(order)
    print(75 * '=')
except:
    pass

In [ ]:
pl = np.array(otb.pl)

In [ ]:
pl

In [ ]:
pl.cumsum()

## Saving the Trading Bot

In [ ]:
import pickle

In [ ]:
agent.model.save('tradingbot')

In [ ]:
agent.model = None

In [ ]:
pickle.dump(agent, open('trading.bot', 'wb'))

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

<br><br><br><a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:ai@tpq.io">ai@tpq.io</a>